# Ejercicio Semana 3

Generar datos sintéticos:

* Escribir modelos que pueden generar datasets
* Utiliza una variedad de modelos y prompts para diversos resultados
* Crea una interfaz de Gradio UI para tu producto

Por qué es útil:

* Se aplica a casi cualquier área de negocios: puede usarlo en su trabajo diario y en proyectos del hogar.